In [1]:
import numpy as np

# 定义模拟参数
num_simulations = 10000  # 蒙特卡洛模拟次数

# 各种配件的次品率(转换为不合格品率)
defective_rate_part1 = 0.20
defective_rate_part2 = 0.20
defective_rate_final_product = 0.20

# 成本参数 (单位: 元)
cost_part1_purchase = 4
cost_part1_inspection = 1
cost_part2_purchase = 18
cost_part2_inspection = 1
cost_assembly = 6
cost_final_inspection = 2
final_product_price = 56

# 不合格品处理成本
replacement_loss = 30  # 调换损失
disassembly_cost = 5   # 拆解费用

# 模拟过程
def simulate_production(inspect_part1, inspect_part2, inspect_final, disassemble):
    # 确定零配件1是否不合格（使用固定的10%概率）
    part1_defective = 0.1 < defective_rate_part1
    part2_defective = 0.1 < defective_rate_part2

    total_cost = cost_part1_purchase + cost_part2_purchase  # 初始采购成本

    if inspect_part1:
        total_cost += cost_part1_inspection
        if part1_defective:
            return total_cost, 0

    if inspect_part2:
        total_cost += cost_part2_inspection
        if part2_defective:
            return total_cost, 0

    # 装配和成品检测成本
    total_cost += cost_assembly
    if inspect_final:
        total_cost += cost_final_inspection

    # 成品是否不合格（次品率受零配件是否合格影响）
    if part1_defective or part2_defective:
        final_product_defective = True
    else:
        final_product_defective = 0.1 < defective_rate_final_product

    if final_product_defective:
        if disassemble:
            # 如果选择拆解，计算拆解后的成本和收益
            disassembly_costs, disassembly_revenue = simulate_production(inspect_part1, inspect_part2, inspect_final, False)
            total_cost += replacement_loss + disassembly_cost + disassembly_costs
            return total_cost, disassembly_revenue
        else:
            # 如果不拆解，直接丢弃
            total_cost += replacement_loss
            return total_cost, 0
    else:
        return total_cost, final_product_price

# 遍历所有策略组合
strategies = [(inspect_part1, inspect_part2, inspect_final, disassemble) 
              for inspect_part1 in [True, False]
              for inspect_part2 in [True, False]
              for inspect_final in [True, False]
              for disassemble in [True, False]]

# 计算每种策略组合的平均成本和收益
for strategy in strategies:
    inspect_part1, inspect_part2, inspect_final, disassemble = strategy
    total_costs = []
    total_revenues = []

    for _ in range(num_simulations):
        cost, revenue = simulate_production(inspect_part1, inspect_part2, inspect_final, disassemble)
        total_costs.append(cost)
        total_revenues.append(revenue)

    avg_cost = np.mean(total_costs)
    avg_revenue = np.mean(total_revenues)
    net_profit = avg_revenue - avg_cost

    print(f"策略组合: 检测零配件1={inspect_part1}, 检测零配件2={inspect_part2}, "
          f"检测成品={inspect_final}, 拆解={disassemble} => "
          f"平均成本: {avg_cost:.2f}, 平均收益: {avg_revenue:.2f}, 平均利润: {net_profit:.2f}")

策略组合: 检测零配件1=True, 检测零配件2=True, 检测成品=True, 拆解=True => 平均成本: 37.75, 平均收益: 32.58, 平均利润: -5.18
策略组合: 检测零配件1=True, 检测零配件2=True, 检测成品=True, 拆解=False => 平均成本: 32.71, 平均收益: 28.71, 平均利润: -4.00
策略组合: 检测零配件1=True, 检测零配件2=True, 检测成品=False, 拆解=True => 平均成本: 36.12, 平均收益: 32.32, 平均利润: -3.80
策略组合: 检测零配件1=True, 检测零配件2=True, 检测成品=False, 拆解=False => 平均成本: 31.59, 平均收益: 28.31, 平均利润: -3.28
策略组合: 检测零配件1=True, 检测零配件2=False, 检测成品=True, 拆解=True => 平均成本: 50.48, 平均收益: 37.21, 平均利润: -13.27
策略组合: 检测零配件1=True, 检测零配件2=False, 检测成品=True, 拆解=False => 平均成本: 37.80, 平均收益: 28.89, 平均利润: -8.91
策略组合: 检测零配件1=True, 检测零配件2=False, 检测成品=False, 拆解=True => 平均成本: 48.06, 平均收益: 37.30, 平均利润: -10.76
策略组合: 检测零配件1=True, 检测零配件2=False, 检测成品=False, 拆解=False => 平均成本: 36.50, 平均收益: 28.55, 平均利润: -7.95
策略组合: 检测零配件1=False, 检测零配件2=True, 检测成品=True, 拆解=True => 平均成本: 50.31, 平均收益: 36.72, 平均利润: -13.59
策略组合: 检测零配件1=False, 检测零配件2=True, 检测成品=True, 拆解=False => 平均成本: 38.18, 平均收益: 28.69, 平均利润: -9.49
策略组合: 检测零配件1=False, 检测零配件2=True, 检测成品=False, 拆解=True => 平均成本: 